# 1. 라이브러리 설치

In [ ]:
!pip install gensim

- colab의 기본 제공 numpy 버젼은 2.0.2
- 2025.4월 기준, 최신 버젼의 gensim은 numpy 1.26.4 버젼을 사용중
1. gensim 설치 과정에서 numpy 버젼이 1.26.4로 조정되었음
2. 그러나, 현재 실행중인 세션 환경에서는 여전히 numpy 버젼이 2.0.2
3. 런타임 > `세션 다시 시작` 을 통해 실행중인 numpy 버젼을 1.26.4 로 업데이트 필요
4. numpy version 확인

- `세션 다시 시작` 전에 numpy 버젼을 확인하여도 1.26.4 버젼이 확인 되지만 실제 세션에서는 2.0.2 버젼을 사용하고 있으므로 반드시 `세션 다시 시작`을 실행해 주세요.


In [ ]:
!pip list | grep numpy

In [ ]:
!pip install konlpy

# 2. 필요한 라이브러리 import

In [ ]:
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt  # 형태소 분석기
# 문자열 정리를 위해 정규 표현식 라이브러리 사용
import re 

# 3. 데이터 수집

- 대한민국 헌법.txt 파일 업로드

In [ ]:
with open("대한민국헌법.txt", encoding='cp949') as f:
	content = f.read()

print(len(content)) # 19063글자로 이루어진 문서

# 4. 데이터 전처리 - 문장에서 불필요한 문자 제거

In [ ]:
# 데이터 정리
print("[정리 전]\n", content[:100])

# 문장별로 분리하기 (한글 기준, '.', '?', '!' 뒤에 공백이 오거나 개행이 오는 경우 분리)
docs = re.split(r'(?<=[.?!])\s+', content.strip())

# 정규 표현식을 통한 한글 외 문자 제거
docs = [re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", doc) for doc in docs]
print("-"*50)
print("[정리 후]\n", docs[:5]) # 정리 후 5개 문장 확인

# 5. 데이터 전처리 - 문장을 단어로 분리(토큰화)

In [ ]:
# 불용어(분석에 도움이 되지 않는 단어들) 목록을 미리 정의
# 예: 조사(의, 가, 이, 는, 등), 자주 등장하지만 의미가 적은 단어(좀, 잘 등)
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# 형태소 분석기 OKT를 사용하여 문장들을 형태소(단어 단위)로 나눌 준비
okt = Okt()

# 최종 토큰(단어)들을 모아둘 리스트
tokenized_data = []
# 각 문장(docs의 원소)에 대해 반복하면서 토큰으로 만듦
for sentence in (docs):
    # okt.morphs: 문장을 형태소(단어) 단위로 나눔. 
    # stem=True: 어간 추출(원형 복원) 수행
    tokenized_sentence = okt.morphs(sentence, stem=True)
    # 불용어 목록에 있는 단어들을 제거
    sentence = [word for word in tokenized_sentence if not word in stopwords]
    # 정제된 단어 리스트를 tokenized_data에 추가
    tokenized_data.append(sentence)
print(tokenized_data)

# 6. Word2Vec Model을 활용
- 한글 토큰을 100차원의 워드 임베딩 벡터로 변환

In [ ]:

# sentences: 학습에 사용할 토큰화된 문장들
# vector_size: 단어 벡터 차원 크기(100차원)
# window: 주변 단어를 고려하는 윈도우 크기(5)
# min_count: 최소 등장 횟수(1회 미만이면 무시)
# workers: 병렬 작업 쓰레드 수(4)
# sg: 0이면 CBOW 방식, 1이면 Skip-Gram 방식
model = Word2Vec(
    sentences = tokenized_data,
    vector_size = 100,
    window = 5,
    min_count = 1,
    workers = 4,
    sg = 0
)

In [ ]:

# 가장 많은 빈도수를 보이는 단어 50개 출력
print(model.wv.index_to_key[:50])

# 7. 불용어 정의 업데이트 및 코드 재 실행 후, 결과 확인​ 

In [ ]:
# '을' 추가
stopwords = ['을', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
tokenized_data = []
for sentence in (docs):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    sentence = [word for word in tokenized_sentence if not word in stopwords]
    tokenized_data.append(sentence)

model = Word2Vec(
    sentences = tokenized_data,
    vector_size = 100,
    window = 5,
    min_count = 1,
    workers = 4,
    sg = 0
)

In [ ]:
# 가장 많은 빈도수를 보이는 단어 50개 출력
print(model.wv.index_to_key[:50])

In [ ]:
# "법률"이라는 단어가 학습된 100차원 벡터를 확인
print(model.wv.get_vector("법률"))
print(len(model.wv.get_vector("법률")))

# 7. 코싸인 유사도 함수 헌법 및 적용

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 코사인 유사도 함수
def cosine_similarity(A, B):
    return dot(A, B) / (norm(A) * norm(B))

# '법률'단어 벡터와 '헌법' 단어 벡터 간의 코사인 유사도 계산
similarity_value = cosine_similarity(model.wv.get_vector('법률'), model.wv.get_vector('헌법')                        )
print("'법률'과 '헌법'의 코사인 유사도:", similarity_value)  # 0.7452134

In [ ]:

# 학습된 벡터 중, '헌법'과 가장 유사도가 높은 순으로 10개 단어를 출력
most_similar_words = model.wv.most_similar('헌법')
print("헌법과 유사한 단어 Top 10:", most_similar_words)